In [31]:

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(",")
        return list(map(str.strip, items))
        

p = CommaOutputParser()

p.parse("Hello, how, are, you ")


['Hello', 'how', 'are', 'you']

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

#temperature 란?
#사용할 샘플링 온도는 0에서 2 사이입니다. 0.8과 같이 값이 높을수록 출력이 더 무작위로 만들어지고, 0.2와 같이 값이 낮을수록 더 집중적이고 결정적이게 됩니다.
chat = ChatOpenAI(
    temperature=0.1 
    )

template = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Korean",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
    )

chat.predict_messages(prompt)



AIMessage(content='Καλησπέρα! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [37]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system",
    "you are a list generating machine. Everything you are asked will be answered with a comma separated of max {max_items}. Do Not reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items="10",
    question="What are you planets",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [38]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":"5",
    "question":"What are you pokenmons?",
})



['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle']

In [14]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Korean",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
    )

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')